In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
testing_length = 40

full_angle_recon = np.load('full_angle_ic_exp_1600_7th_recons_weight_0.25_clip_True.npy')[-testing_length:, -1, 0, 11:-11, 11:-11].astype(np.float32)

offset = 50

FBP = []
iterative_sparse_recon = []
iterative_sparse_TV_recon = []

for photon in tqdm([32, 40, 50, 64, 80, 100, 128, 160, 200, 256, 
                    320, 400, 500, 640, 800, 1000, 1280, 1600, 2000]):
    
    temp = np.load('./FBP_results/FDK-ic-1600-exp-test-photon-' + str(photon) + '-offset-' + str(offset) + '.npy')[-testing_length:, 11:-11, 11:-11]
    FBP.append(temp)
    
    temp = np.load('./iterative_results/iterative_sparse_exp_7th_photon_' + str(photon) + '_offset_' + str(offset) + '_weight_0_clip_True.npy')[-testing_length:, -1, 0, 11:-11, 11:-11]
    iterative_sparse_recon.append(temp)
    
    temp = np.load('./iterative_results/iterative_sparse_exp_7th_photon_' + str(photon) + '_offset_' + str(offset) + '_weight_2_clip_True.npy')[-testing_length:, -1, 0, 11:-11, 11:-11]
    iterative_sparse_TV_recon.append(temp)
    
FBP = (np.concatenate(FBP, axis=0)).astype(np.float32)
iterative_sparse_recon = (np.concatenate(iterative_sparse_recon, axis=0))
iterative_sparse_TV_recon = (np.concatenate(iterative_sparse_TV_recon, axis=0))

FBP_UNet0 = np.load('./FBP_UNet_results/exp_FBP_UNets_model_0_offset_' + str(offset) + '_poisson_32to2000.npy')[:, :, :, 0]

iter10 = np.load('./FBP_UNet_results/exp_starting_iter_10_UNets_model_0_offset_' + str(offset) + '_weight_0_clip_True_poisson_32to2000.npy')[:, :, :, 0]
iter20 = np.load('./FBP_UNet_results/exp_starting_iter_20_UNets_model_0_offset_' + str(offset) + '_weight_0_clip_True_poisson_32to2000.npy')[:, :, :, 0]
iter30 = np.load('./FBP_UNet_results/exp_starting_iter_30_UNets_model_0_offset_' + str(offset) + '_weight_0_clip_True_poisson_32to2000.npy')[:, :, :, 0]
iter40 = np.load('./FBP_UNet_results/exp_starting_iter_40_UNets_model_0_offset_' + str(offset) + '_weight_0_clip_True_poisson_32to2000.npy')[:, :, :, 0]
iter50 = np.load('./FBP_UNet_results/exp_starting_iter_50_UNets_model_0_offset_' + str(offset) + '_weight_0_clip_True_poisson_32to2000.npy')[:, :, :, 0]

iter10_TV = np.load('./FBP_UNet_results/exp_starting_iter_10_UNets_model_0_offset_' + str(offset) + '_weight_2_clip_True_poisson_32to2000.npy')[:, :, :, 0]
iter20_TV = np.load('./FBP_UNet_results/exp_starting_iter_20_UNets_model_0_offset_' + str(offset) + '_weight_2_clip_True_poisson_32to2000.npy')[:, :, :, 0]
iter30_TV = np.load('./FBP_UNet_results/exp_starting_iter_30_UNets_model_0_offset_' + str(offset) + '_weight_2_clip_True_poisson_32to2000.npy')[:, :, :, 0]
iter40_TV = np.load('./FBP_UNet_results/exp_starting_iter_40_UNets_model_0_offset_' + str(offset) + '_weight_2_clip_True_poisson_32to2000.npy')[:, :, :, 0]
iter50_TV = np.load('./FBP_UNet_results/exp_starting_iter_50_UNets_model_0_offset_' + str(offset) + '_weight_2_clip_True_poisson_32to2000.npy')[:, :, :, 0]

 63%|██████▎   | 12/19 [01:47<01:00,  8.71s/it]

In [ ]:
# array to make the input and output zeros on the edges
Xs, Ys = np.mgrid[:128,:128]
Xs = Xs - np.mean(Xs)
Ys = Ys - np.mean(Ys)
Rs = np.sqrt(Xs**2 + Ys**2)

r = 71
#r = 100

full_angle_recon[:, Rs>r] = 0
FBP[:, Rs>r] = 0
iterative_sparse_recon[:, Rs>r] = 0
iterative_sparse_TV_recon[:, Rs>r] = 0


FBP_UNet0[:, Rs>r] = 0

iter10[:, Rs>r] = 0
iter20[:, Rs>r] = 0
iter30[:, Rs>r] = 0
iter40[:, Rs>r] = 0
iter50[:, Rs>r] = 0

iter10_TV[:, Rs>r] = 0
iter20_TV[:, Rs>r] = 0
iter30_TV[:, Rs>r] = 0
iter40_TV[:, Rs>r] = 0
iter50_TV[:, Rs>r] = 0

In [ ]:
def compare_pcc(im_true_array, im_test_array):
    
    loss_list = []
    
    for im_true, im_test in zip(im_true_array, im_test_array):
        temp = PCC(im_true, im_test)
        loss_list.append(temp)
        
    result = np.sum(loss_list) / np.shape(loss_list)[0]
    
    return result

def PCC(im_1, im_2):
    
    fsp = im_1 - np.mean(im_1)
    fst = im_2 - np.mean(im_1)
        
    devP = np.std(im_1)
    devT = np.std(im_2)
    
    loss = np.mean(fsp * fst) / \
          np.clip(devP * devT, a_min=1e-10, a_max=None)
    #loss = np.mean(np.square(im_true - im_test))
    return loss

FBP_pcc = []
iterative_sparse_recon_pcc = []
iterative_sparse_TV_recon_pcc =[]

FBP_UNet0_pcc = []

iter10_pcc = []
iter20_pcc = []
iter30_pcc = []
iter40_pcc = []
iter50_pcc = []

iter10_TV_pcc = []
iter20_TV_pcc = []
iter30_TV_pcc = []
iter40_TV_pcc = []
iter50_TV_pcc = []

for idx, photon in tqdm(enumerate([32, 40, 50, 64, 80, 100, 128, 160, 200, 256, 320, 400, 500, 640, 800, 1000, 1280, 1600, 2000])):

    data_temp = FBP[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    FBP_pcc.append(pcc)
    
    data_temp = iterative_sparse_recon[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iterative_sparse_recon_pcc.append(pcc)
    
    data_temp = iterative_sparse_TV_recon[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iterative_sparse_TV_recon_pcc.append(pcc)
    
    data_temp = FBP_UNet0[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    FBP_UNet0_pcc.append(pcc)
    
    data_temp = iter10[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iter10_pcc.append(pcc)
    
    data_temp = iter20[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iter20_pcc.append(pcc)
    
    data_temp = iter30[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iter30_pcc.append(pcc)
    
    data_temp = iter40[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iter40_pcc.append(pcc)
    
    data_temp = iter50[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iter50_pcc.append(pcc)
    
    data_temp = iter10_TV[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iter10_TV_pcc.append(pcc)
    
    data_temp = iter20_TV[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iter20_TV_pcc.append(pcc)
    
    data_temp = iter30_TV[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iter30_TV_pcc.append(pcc)
    
    data_temp = iter40_TV[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iter40_TV_pcc.append(pcc)
    
    data_temp = iter50_TV[testing_length*idx:testing_length*idx+testing_length]
    pcc = compare_pcc(full_angle_recon, data_temp)
    iter50_TV_pcc.append(pcc)

In [ ]:
plt.rcParams['figure.figsize'] = [10*1.2, 8*1.2]
plt.style.use('seaborn-whitegrid')
plt.rcParams["font.family"] = "Mishafi"
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'light'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

plt.rcParams["axes.edgecolor"] = 'black'
plt.rcParams["axes.linewidth"]  = 1.5

# plt.rcParams.update({
#     'font.family': 'serif',
#     'font.serif': 'Times New Roman',  # 'Nimbus Roman No9 L',
# })

markersize = 160

fig, ax = plt.subplots()

plt.scatter(np.arange(19), iter50_TV_pcc, marker='>', s=markersize, color='orange', label="Sparsity regularized UNet")
plt.scatter(np.arange(19), iter50_pcc, marker='d', s=markersize, color='brown', label="Data fidelity UNet")
plt.scatter(np.arange(19)+0.05, FBP_UNet0_pcc, marker='D', s=markersize-20, color='blue', label="FBP UNet")
plt.scatter(np.arange(19)-0.025, iterative_sparse_TV_recon_pcc, marker='o', s=markersize, color='green', label="iterative sparse TV")
plt.scatter(np.arange(19)+0.025, iterative_sparse_recon_pcc, marker='x', s=markersize+80, color='red', label="iterative sparse")
plt.scatter(np.arange(19)-0.05, FBP_pcc, marker='^', color='black', s=markersize, label="FBP")

plt.legend(fontsize=16, loc='lower right')
plt.xlabel("number of photons", fontsize=23)
plt.ylabel("Pearson correlation coefficient", fontsize=23)
plt.xticks(np.arange(19), [32, 40, 50, 64, 80, 100, 128, 160, 200, 256, 320, 400, 500, 640, 800, 1000, 1280, 1600, 2000], rotation=30, fontsize=18)
plt.yticks(fontsize=20)
#plt.ticklabel_format(axis='y', scilimits=(-1, 1), style='sci')
ax.yaxis.offsetText.set_fontsize(20)
plt.ylim(-0.1, 1.05)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10*1.2, 8*1.2]
plt.style.use('seaborn-whitegrid')
plt.rcParams["font.family"] = "Mishafi"
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'light'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

plt.rcParams["axes.edgecolor"] = 'black'
plt.rcParams["axes.linewidth"]  = 1.5

# plt.rcParams.update({
#     'font.family': 'serif',
#     'font.serif': 'Times New Roman',  # 'Nimbus Roman No9 L',
# })

markersize = 160

fig, ax = plt.subplots()

plt.scatter(np.arange(19)-0.05,[np.log(1-ele) for ele in FBP_pcc], marker='^', color='black', s=markersize, label="FBP")
plt.scatter(np.arange(19)+0.025, [np.log(1-ele) for ele in iterative_sparse_recon_pcc], marker='x', s=markersize+80, color='red', label="iterative sparse")
plt.scatter(np.arange(19)-0.025, [np.log(1-ele) for ele in iterative_sparse_TV_recon_pcc], marker='o', s=markersize, color='green', label="iterative sparse TV")
plt.scatter(np.arange(19)+0.05, [np.log(1-ele) for ele in FBP_UNet0_pcc], marker='D', s=markersize-20, color='blue', label="FBP UNet")
plt.scatter(np.arange(19), [np.log(1-ele) for ele in iter50_pcc], marker='d', s=markersize, color='brown', label="Data fidelity UNet")
plt.scatter(np.arange(19), [np.log(1-ele) for ele in iter50_TV_pcc], marker='>', s=markersize, color='orange', label="Sparsity regularized UNet")

plt.legend(fontsize=18, loc='lower left')
plt.xlabel("number of photons", fontsize=23)
plt.ylabel("log of 1 - Pearson correlation coefficient", fontsize=21)
plt.xticks(np.arange(19), [32, 40, 50, 64, 80, 100, 128, 160, 200, 256, 320, 400, 500, 640, 800, 1000, 1280, 1600, 2000], rotation=30, fontsize=18)
plt.yticks(fontsize=17)
#plt.ticklabel_format(axis='y', scilimits=(-1, 1), style='sci')
ax.yaxis.offsetText.set_fontsize(20)
#plt.ylim(0.895, 1.003)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10*1.2, 8*1.2]
plt.style.use('seaborn-whitegrid')
plt.rcParams["font.family"] = "Mishafi"
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'light'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

plt.rcParams["axes.edgecolor"] = 'black'
plt.rcParams["axes.linewidth"]  = 1.5

# plt.rcParams.update({
#     'font.family': 'serif',
#     'font.serif': 'Times New Roman',  # 'Nimbus Roman No9 L',
# })

markersize = 160

fig, ax = plt.subplots()

plt.scatter(np.arange(19), iter50_pcc, marker='d', s=markersize, color='brown', label="Data fidelity UNet (50 iters)")
plt.scatter(np.arange(19)-0.025, iter40_pcc, marker='o', s=markersize, color='green', label="Data fidelity UNet (40 iters)")
plt.scatter(np.arange(19), iter30_pcc, marker='>', s=markersize, color='orange', label="Data fidelity UNet (30 iters)")
#plt.scatter(np.arange(19), iter25_pcc, marker='d', s=markersize, color='brown', label="Non-negativity UNet (25 iters)")
plt.scatter(np.arange(19)+0.05, iter20_pcc, marker='D', s=markersize-20, color='blue', label="Data fidelity UNet (20 iters)")
#plt.scatter(np.arange(19)-0.025, iter15_pcc, marker='o', s=markersize, color='green', label="Non-negativity UNet (15 iters)")
#plt.scatter(np.arange(19)+0.025, iter10_pcc, marker='x', s=markersize+80, color='red', label="Data fidelity UNet (10 iters)")
#plt.scatter(np.arange(19)-0.05, iter5_pcc, marker='^', color='black', s=markersize, label="Non-negativity UNet (5 iters)")

plt.legend(fontsize=18, loc='lower right')
plt.xlabel("number of photons", fontsize=23)
plt.ylabel("Pearson correlation coefficient", fontsize=23)
plt.xticks(np.arange(19), [32, 40, 50, 64, 80, 100, 128, 160, 200, 256, 320, 400, 500, 640, 800, 1000, 1280, 1600, 2000], rotation=30, fontsize=18)
plt.yticks(fontsize=20)
#plt.ticklabel_format(axis='y', scilimits=(-1, 1), style='sci')
ax.yaxis.offsetText.set_fontsize(20)
plt.ylim(0.5, 1.02)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10*1.2, 8*1.2]
plt.style.use('seaborn-whitegrid')
plt.rcParams["font.family"] = "Mishafi"
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'light'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

plt.rcParams["axes.edgecolor"] = 'black'
plt.rcParams["axes.linewidth"]  = 1.5

# plt.rcParams.update({
#     'font.family': 'serif',
#     'font.serif': 'Times New Roman',  # 'Nimbus Roman No9 L',
# })

markersize = 160

fig, ax = plt.subplots()

plt.scatter(np.arange(19), [np.log(1-ele) for ele in iter50_pcc], marker='d', s=markersize, color='brown', label="Data fidelity UNet (50 iters)")
plt.scatter(np.arange(19)-0.025, [np.log(1-ele) for ele in iter40_pcc], marker='o', s=markersize, color='green', label="Data fidelity UNet (40 iters)")
plt.scatter(np.arange(19), [np.log(1-ele) for ele in iter30_pcc], marker='>', s=markersize, color='orange', label="Data fidelity UNet (30 iters)")
#plt.scatter(np.arange(19), iter25_pcc, marker='d', s=markersize, color='brown', label="Non-negativity UNet (25 iters)")
plt.scatter(np.arange(19)+0.05, [np.log(1-ele) for ele in iter20_pcc], marker='D', s=markersize-20, color='blue', label="Data fidelity UNet (20 iters)")
#plt.scatter(np.arange(19)-0.025, iter15_pcc, marker='o', s=markersize, color='green', label="Non-negativity UNet (15 iters)")
#plt.scatter(np.arange(19)+0.025, iter10_pcc, marker='x', s=markersize+80, color='red', label="Data fidelity UNet (10 iters)")
#plt.scatter(np.arange(19)-0.05, iter5_pcc, marker='^', color='black', s=markersize, label="Non-negativity UNet (5 iters)")

plt.legend(fontsize=18, loc='upper right')
plt.xlabel("number of photons", fontsize=23)
plt.ylabel("log of 1 - Pearson correlation coefficient", fontsize=21)
plt.xticks(np.arange(19), [32, 40, 50, 64, 80, 100, 128, 160, 200, 256, 320, 400, 500, 640, 800, 1000, 1280, 1600, 2000], rotation=30, fontsize=18)
plt.yticks(fontsize=20)
#plt.ticklabel_format(axis='y', scilimits=(-1, 1), style='sci')
ax.yaxis.offsetText.set_fontsize(20)
plt.ylim(-6.8, 0.3)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10*1.2, 8*1.2]
plt.style.use('seaborn-whitegrid')
plt.rcParams["font.family"] = "Mishafi"
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'light'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

plt.rcParams["axes.edgecolor"] = 'black'
plt.rcParams["axes.linewidth"]  = 1.5

# plt.rcParams.update({
#     'font.family': 'serif',
#     'font.serif': 'Times New Roman',  # 'Nimbus Roman No9 L',
# })

markersize = 120

fig, ax = plt.subplots()

plt.scatter(np.arange(19), iter50_TV_pcc, marker='d', s=markersize, color='brown', label="Sparsity regularized UNet (50 iters)")
plt.scatter(np.arange(19)-0.025, iter40_TV_pcc, marker='o', s=markersize, color='green', label="Sparsity regularized UNet (40 iters)")
plt.scatter(np.arange(19), iter30_TV_pcc, marker='>', s=markersize, color='orange', label="Sparsity regularized UNet (30 iters)")
#plt.scatter(np.arange(19), iter25_TV_pcc, marker='d', s=markersize, color='brown', label="Sparsity regularized UNet (25 iters)")
plt.scatter(np.arange(19)+0.05, iter20_TV_pcc, marker='D', s=markersize-20, color='blue', label="Sparsity regularized UNet (20 iters)")
#plt.scatter(np.arange(19)-0.025, iter15_TV_pcc, marker='o', s=markersize, color='green', label="Sparsity regularized UNet (15 iters)")
#plt.scatter(np.arange(19)+0.025, iter10_TV_pcc, marker='x', s=markersize+80, color='red', label="Sparsity regularized UNet (10 iters)")
#plt.scatter(np.arange(19)-0.05, iter5_TV_pcc, marker='^', color='black', s=markersize, label="Sparsity regularized UNet (5 iters)")

plt.legend(fontsize=18, loc='lower right')
plt.xlabel("number of photons", fontsize=23)
plt.ylabel("Pearson correlation coefficient", fontsize=23)
plt.xticks(np.arange(19), [32, 40, 50, 64, 80, 100, 128, 160, 200, 256, 320, 400, 500, 640, 800, 1000, 1280, 1600, 2000], rotation=30, fontsize=18)
plt.yticks(fontsize=20)
#plt.ticklabel_format(axis='y', scilimits=(-1, 1), style='sci')
ax.yaxis.offsetText.set_fontsize(20)
plt.ylim(0.5, 1.02)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10*1.2, 8*1.2]
plt.style.use('seaborn-whitegrid')
plt.rcParams["font.family"] = "Mishafi"
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'light'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

plt.rcParams["axes.edgecolor"] = 'black'
plt.rcParams["axes.linewidth"]  = 1.5

# plt.rcParams.update({
#     'font.family': 'serif',
#     'font.serif': 'Times New Roman',  # 'Nimbus Roman No9 L',
# })

markersize = 120

fig, ax = plt.subplots()

plt.scatter(np.arange(19), [np.log(1-ele) for ele in iter50_TV_pcc], marker='d', s=markersize, color='brown', label="Sparsity regularized UNet (50 iters)")
plt.scatter(np.arange(19)-0.025, [np.log(1-ele) for ele in iter40_TV_pcc], marker='o', s=markersize, color='green', label="Sparsity regularized UNet (40 iters)")
plt.scatter(np.arange(19), [np.log(1-ele) for ele in iter30_TV_pcc], marker='>', s=markersize, color='orange', label="Sparsity regularized UNet (30 iters)")
#plt.scatter(np.arange(19), iter25_TV_pcc, marker='d', s=markersize, color='brown', label="Sparsity regularized UNet (25 iters)")
plt.scatter(np.arange(19)+0.05, [np.log(1-ele) for ele in iter20_TV_pcc], marker='D', s=markersize-20, color='blue', label="Sparsity regularized UNet (20 iters)")
#plt.scatter(np.arange(19)-0.025, iter15_TV_pcc, marker='o', s=markersize, color='green', label="Sparsity regularized UNet (15 iters)")
#plt.scatter(np.arange(19)+0.025, iter10_TV_pcc, marker='x', s=markersize+80, color='red', label="Sparsity regularized UNet (10 iters)")
#plt.scatter(np.arange(19)-0.05, iter5_TV_pcc, marker='^', color='black', s=markersize, label="Sparsity regularized UNet (5 iters)")

plt.legend(fontsize=18, loc='upper right')
plt.xlabel("number of photons", fontsize=23)
plt.ylabel("log of 1 - Pearson correlation coefficient", fontsize=21)
plt.xticks(np.arange(19), [32, 40, 50, 64, 80, 100, 128, 160, 200, 256, 320, 400, 500, 640, 800, 1000, 1280, 1600, 2000], rotation=30, fontsize=18)
plt.yticks(fontsize=20)
#plt.ticklabel_format(axis='y', scilimits=(-1, 1), style='sci')
ax.yaxis.offsetText.set_fontsize(20)
plt.ylim(-6.8, 0.3)
plt.show()

In [ ]:
obj_idx = 35

offset1 = 0 + 2
offset2 = 4 + 2
offset3 = 8 + 2
offset4 = 12 + 2
offset5 = 16 + 2

x = 5
y = 6

plt.style.use('seaborn-whitegrid')

plt.rcParams['figure.figsize'] = [50.3, 60.6]
fig = plt.figure()
fig.subplots_adjust(hspace=.01, wspace=.01)

for i in range(x*y):
     
    ax = fig.add_subplot(y, x, i +1)
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    
    if i//x == 5 :
        if i % x == 0:
            plt.imshow(np.rot90(iter50_TV[obj_idx + testing_length*offset1], 3), cmap='viridis')
        elif i % x == 1:
            plt.imshow(np.rot90(iter50_TV[obj_idx + testing_length*offset2], 3), cmap='viridis')
        elif i % x == 2:
            plt.imshow(np.rot90(iter50_TV[obj_idx + testing_length*offset3], 3), cmap='viridis')
        elif i % x == 3:
            plt.imshow(np.rot90(iter50_TV[obj_idx + testing_length*offset4], 3), cmap='viridis')
        elif i % x == 4:
            plt.imshow(np.rot90(iter50_TV[obj_idx + testing_length*offset5], 3), cmap='viridis')
        plt.clim(0, 0.1)
    
    if i//x == 4 :
        if i % x == 0:
            plt.imshow(np.rot90(iter50[obj_idx + testing_length*offset1], 3), cmap='viridis')
        elif i % x == 1:
            plt.imshow(np.rot90(iter50[obj_idx + testing_length*offset2], 3), cmap='viridis')
        elif i % x == 2:
            plt.imshow(np.rot90(iter50[obj_idx + testing_length*offset3], 3), cmap='viridis')
        elif i % x == 3:
            plt.imshow(np.rot90(iter50[obj_idx + testing_length*offset4], 3), cmap='viridis')
        elif i % x == 4:
            plt.imshow(np.rot90(iter50[obj_idx + testing_length*offset5], 3), cmap='viridis')
        plt.clim(0, 0.1)
    if i//x == 3 :
        if i % x == 0:
            plt.imshow(np.rot90(FBP_UNet0[obj_idx + testing_length*offset1], 3), cmap='viridis')
        elif i % x == 1:
            plt.imshow(np.rot90(FBP_UNet0[obj_idx + testing_length*offset2], 3), cmap='viridis')
        elif i % x == 2:
            plt.imshow(np.rot90(FBP_UNet0[obj_idx + testing_length*offset3], 3), cmap='viridis')
        elif i % x == 3:
            plt.imshow(np.rot90(FBP_UNet0[obj_idx + testing_length*offset4], 3), cmap='viridis')
        elif i % x == 4:
            plt.imshow(np.rot90(FBP_UNet0[obj_idx + testing_length*offset5], 3), cmap='viridis')
        plt.clim(0, 0.1)
    if i//x == 2 :
        if i % x == 0:
            plt.imshow(np.rot90(iterative_sparse_TV_recon[obj_idx + testing_length*offset1], 3), cmap='viridis')
        elif i % x == 1:
            plt.imshow(np.rot90(iterative_sparse_TV_recon[obj_idx + testing_length*offset2], 3), cmap='viridis')
        elif i % x == 2:
            plt.imshow(np.rot90(iterative_sparse_TV_recon[obj_idx + testing_length*offset3], 3), cmap='viridis')
        elif i % x == 3:
            plt.imshow(np.rot90(iterative_sparse_TV_recon[obj_idx + testing_length*offset4], 3), cmap='viridis')
        elif i % x == 4:
            plt.imshow(np.rot90(iterative_sparse_TV_recon[obj_idx + testing_length*offset5], 3), cmap='viridis')
        plt.clim(0, 0.1)
    if i//x == 1 :
        if i % x == 0:
            plt.imshow(np.rot90(iterative_sparse_recon[obj_idx + testing_length*offset1], 3), cmap='viridis')
        elif i % x == 1:
            plt.imshow(np.rot90(iterative_sparse_recon[obj_idx + testing_length*offset2], 3), cmap='viridis')
        elif i % x == 2:
            plt.imshow(np.rot90(iterative_sparse_recon[obj_idx + testing_length*offset3], 3), cmap='viridis')
        elif i % x == 3:
            plt.imshow(np.rot90(iterative_sparse_recon[obj_idx + testing_length*offset4], 3), cmap='viridis')
        elif i % x == 4:
            plt.imshow(np.rot90(iterative_sparse_recon[obj_idx + testing_length*offset5], 3), cmap='viridis')
        plt.clim(0, 0.1)
    if i//x == 0 :
        if i % x == 0:
            plt.imshow(np.rot90(FBP[obj_idx + testing_length*offset1], 3), cmap='viridis')
        elif i % x == 1:
            plt.imshow(np.rot90(FBP[obj_idx + testing_length*offset2], 3), cmap='viridis')
        elif i % x == 2:
            plt.imshow(np.rot90(FBP[obj_idx + testing_length*offset3], 3), cmap='viridis')
        elif i % x == 3:
            plt.imshow(np.rot90(FBP[obj_idx + testing_length*offset4], 3), cmap='viridis')
        elif i % x == 4:
            plt.imshow(np.rot90(FBP[obj_idx + testing_length*offset5], 3), cmap='viridis')
        plt.clim(0, 0.1)